In [1]:
import pickle
import datetime
import pandas.io.data as web
import sys
import pandas as pd
sys.path.append('../prod/')
from Portfolio import Portfolio, Position
import os

In [18]:
start_date = datetime.datetime(2016,10,3)

In [20]:
P1 = (['FB', 'GLD', 'GPS', 'OIL', 'XOM'],[0.2,0.2,0.2,0.2,0.2],[127.8,124.9,21.57,5.83,86.54])
p = start_manager_update_process(P1)

###Manager Updates

In [19]:
#where p is just like P1 : (tickers,initial weights, price of purchase)
def start_manager_update_process(p):
    if os.path.isfile('./portfolio.txt') is False:
        # a  few requirements to set up the data pipeline
        df = web.DataReader(['GOOGL','GLD'],'yahoo',datetime.datetime(2010, 1, 1))['Close']
        df = df.pct_change()[1:len(df)]
        df.columns = ['GOOGL','GLD']
        df.to_csv("returns_data.csv")
        with open('portfolio.txt','wb') as f:
            pickle.dump([],f)
  
    latest_portfolio = manager_portfolio_update(p)
    save_new_portfolio(latest_portfolio)
    return latest_portfolio

def manager_portfolio_update(p):
    tickers = p[0]
    weights = p[1]
    costs = p[2]
    df = manager_data_update(tickers)
    df = df.dropna(thresh=len(tickers), axis=0) #this line restrict historical calculations to data points shared by all equities
    all_dates = df.index
    assets = []
    for i in range(len(tickers)):
        symbol = tickers[i]
        w = weights[i]
        in_price = costs[i]
        returns = df[symbol]
        direction_pos = direction(w)
        assets.append(Position(symbol,returns,direction_pos,in_price,w))
    #start_date = datetime.date.today() - datetime.timedelta(3)
    start_date = datetime.datetime(2016,10,3)
    latest_portfolio = Portfolio(assets,start_date,all_dates)
    return latest_portfolio
    
#initially load csv with one series going back at least to 2010
def manager_data_update(tickers):
    df = pd.DataFrame.from_csv("returns_data.csv")
    tickers = set(tickers)
    new_tickers = list(tickers - set(df.columns))
    new_columns = web.DataReader(new_tickers,'yahoo',datetime.datetime(2010, 1, 1))['Close']
    new_columns = new_columns.pct_change()[1:len(new_columns)]
    df = df.join(new_columns,how='outer')
    df.to_csv("returns_data.csv")
    return df

def save_new_portfolio(latest_portfolio):
    all_portfolios = pull_old_portfolios()
    all_portfolios.append(latest_portfolio.compile_portfolio())
    with open('portfolio.txt', 'wb') as f:
        pickle.dump(all_portfolios, f)

###General

In [3]:
def direction(w):
    if w >= 0:
        return "long"
    else:
        return "short"

#use this to plot the historical performance of the fund, for whatever asset allocation
def pull_old_portfolios():
    with open('portfolio.txt','rb') as f:
        var = pickle.load(f)
    return var

### Daily Updates

In [4]:
def start_daily_update_process():
    latest_saved_portfolio = pull_latest_portfolio()
    df = daily_data_update()
    latest_portfolio = daily_portfolio_update(latest_saved_portfolio,df)
    update_last_portfolio(latest_portfolio)

def daily_portfolio_update(latest_saved_portfolio,return_grid):
    df = return_grid
    latest_portfolio = latest_saved_portfolio.uncompile_and_update(df)
    return latest_portfolio
    
def pull_latest_portfolio():
    olds = pull_old_portfolios()
    return olds[len(olds)-1]

def daily_data_update():
    df = pd.DataFrame.from_csv("returns_data.csv")
    tickers = df.columns
    today_row = web.DataReader(tickers,'yahoo',start_date,datetime.date.today())['Close']
    today_row = today_row.pct_change()[1:len(today_row)]
    df = df.append(today_row)
    df.to_csv("returns_data.csv")
    return df

def update_last_portfolio(latest_portfolio):
    all_portfolios = pull_old_portfolios()
    all_portfolios[len(all_portfolios)-1] = latest_portfolio.compile_portfolio()
    with open('portfolio.txt', 'wb') as f:
        pickle.dump(all_portfolios, f)

In [21]:
start_daily_update_process()

In [22]:
port = pull_latest_portfolio()

In [23]:
port

In [24]:
port = port.uncompile()

In [27]:
port.daily_values

[5.9019330961499641,
 5.7112310537418916,
 3.7056248513536598,
 9.173448264667563,
 15.712082868372507,
 54.095796274634914,
 152.04114585827747,
 66.146580934036123,
 61.869458156439705,
 19.989881512484583,
 18.987953016605072,
 11.799345165723254,
 18.286830433152659,
 42.898360836552811,
 18.513600199879424,
 25.016515153557744,
 37.806893659617408,
 9.8607713531614607,
 6.9394914817749491,
 4.1135824428849821,
 3.2825156433896781,
 -1.1197038381500124,
 -0.34266449794942133,
 0.14341684694864251,
 -0.2438676506417361,
 -0.26867559790866619,
 -0.97014582604446375,
 -1.1553593724519777,
 -2.3160421711805688,
 0.24460221178603964,
 0.60734686990404674,
 0.8995645312164261,
 3.1622709063743284,
 8.0465347241616954,
 3.5929674764006383,
 3.279565796451823,
 1.839832420369852,
 1.5184261639258732,
 0.94043754641687893,
 1.2322891598891907,
 2.871490417372291,
 1.4256647274704746,
 1.8633564641038163,
 2.2045585117344957,
 1.2871209257077174,
 0.96209650635741928,
 0.71653426569528489,
 

In [26]:
port.start_date

datetime.datetime(2016, 10, 3, 0, 0)